In [1]:
# uses conda environment c14
import pandas as pd
import numpy as np
from numpy import asarray
import xarray as xr
import pkg_resources
from csv import reader
from math import exp, pow, sqrt
from copy import copy
from openpyxl import load_workbook
from datetime import date
from pathlib import Path



import cartopy.crs as ccrs
import cartopy.feature as cfeature 
from matplotlib.lines import Line2D
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


from iosacal.core import CalAge, CalibrationCurve
from iosacal.core import RadiocarbonDetermination as R
from iosacal.hpd import hpd_interval
import iosacal as ios
from iosacal.text import single_text


from calcDelR import calcDelR

PROJECT_ROOT = Path.cwd().parents[1]
data_dir = PROJECT_ROOT / "ws/data/"
print('done')

done


In [16]:
xls = pd.ExcelFile('HOLSEA_2020_Singapore.xlsx',  engine='openpyxl')
df = pd.read_excel(xls, 'Singapore', header=1).drop(0)

#Make new dataframe for radiocarbon ages
df_c = df[['Latitude', 
           'Longitude',
           'Dated facies',
           'Unique sample ID',
          'Corrected age       (14C a BP)',
          'Corrected age uncertainty (14C a)']].copy()


######## NOTE: THIS IS BRITTLE.  MAKE A COLUMN IN YOUR SPREADSHEET #######
######## THAT LABELS WHICH DATA ARE MARINE VS TERRESTRIAL ################
df_c['ismarine'] = df_c['Dated facies'].map(lambda x: 1 if 'marine' in x else 0)

######## THIS LINE CURRENTLY ADDs 100 YRS BULK UNCERTAINTY TO EACH DATUM ######
######## RANDOMLY AS AN EXAMPLE. TO ADD BULK UNCERTAINTY TO ONLY A CHOSEN PART  #####
######## OF THE DATA, ADD A BINARY COLUMN NAMED BULK THAT SPECIFIES WHICH DATA ARE BULK ###3
######## AND COMMENT THIS OUT ####
df_c['bulk_uncertainty'] = np.round(np.random.random(len(df)))  * 100


# Calculate Delta R

In [17]:
def apply_delR(df, df_R, n, which):
    df['lat'] = df.Latitude
    df['lon'] = df.Longitude
    
    try:
        if 'mu' in which:
            return df.apply(lambda r: calcDelR(df_R, r.Latitude, r.Longitude, n).getMu(), axis=1)
    
        elif 'uncertainty' in which:
            return df.apply(lambda r: calcDelR(df_R, r.Latitude, r.Longitude, n).getUncert(), axis=1)
        
    except ValueError:
        print('Use either mu or uncertainty')


df_R = pd.read_excel('delta_R_calibration.xlsx')
df_R.head(2)
        
        
##### 'n' SPECIFIES HOW MANY OF THE CLOSEST MARINE RESERVOIR VALUES ####
#####  TO EACH SAMPLE ARE AVERAGED, WEIGHTED BY INVERSE DISTANCE ###############
n=4

df_c['ΔR (14C a)'] = apply_delR(df_c, df_R, n, 'mu')
df_c['Reservoir age uncertainty (14C a)'] = apply_delR(df_c, df_R, n, 'uncertainty')

print('done')

done


# Calibrate Radiocarbon ages 

In [19]:
for i, r in df_c.iterrows():

    age =  r['Corrected age       (14C a BP)'] + r['ΔR (14C a)']
    age_onesig = np.sqrt(r['Corrected age uncertainty (14C a)']**2 +
                         r['Reservoir age uncertainty (14C a)']**2 +
                         r.bulk_uncertainty)

    if age < 0:  
        t1 = "This code cannot process dates younger than 1950."
        t2 = "Please remove those dates and rerun."
        raise ValueError(t1 + t2)



    # Calculate radiocarbon determination
    cr = R(age, age_onesig, r['Unique sample ID'])

    # Choose calibration curve.  
    # Note: you can add any .14c file (e.g. Brendryen 2020's normarine18)
    # by searching for 'intcal20.14c', navigating to the directory where 
    # it is stored, and adding the new .14c file.
    calib = 'intcal20'
    if r.ismarine:
        calib = 'marine20'
    elif (r.lat < 0):
        calib = 'shcal20'


    cal =  cr.calibrate(calib)

    # Note: iosacal has a bug in the current version which 
    # incorrectly calculates the median age from the confidence interval
    # This line fixes it, but hopefully will not be necessary in future 
    # iosacal iterations.
    CImed = np.median([(c.from_year, c.to_year) for c in hpd_interval(cal, 0.5)])
    CI95high = cal.intervals[95][0].from_year
    CI95low = cal.intervals[95][-1].to_year

    # Add 
    df_c.loc[i, 'Age                   (cal a BP)'] = CImed
    df_c.loc[i, 'Age 2σ Uncertainty +              (cal a)'] = CI95high - CImed 
    df_c.loc[i, 'Age 2σ Uncertainty -              (cal a)'] = CImed - CI95low


    if i%10 == 0:
        print(f'----{i}----')
        print('corrected age = ', age)
        print('c14 age uncertainty = ', age_onesig)

        print('Calibrated age = ', CImed)
        print('calibrated age error (-,+) =', CImed - CI95low, CI95high - CImed )
        print('------------')




----10----
corrected age =  7828.1635690286585
c14 age uncertainty =  71.86938640672918
Calibrated age =  8684.5
calibrated age error (-,+) = 259.5 294.5
------------
----20----
corrected age =  6558.1635690286585
c14 age uncertainty =  96.25075949040479
Calibrated age =  7525.5
calibrated age error (-,+) = 253.5 77.5
------------
----30----
corrected age =  7548.1635690286585
c14 age uncertainty =  113.42049507245041
Calibrated age =  8285.5
calibrated age error (-,+) = 242.5 303.5
------------
----40----
corrected age =  6867.1635690286585
c14 age uncertainty =  68.65281277908247
Calibrated age =  7647.0
calibrated age error (-,+) = 68.0 196.0
------------
----50----
corrected age =  7398.1635690286585
c14 age uncertainty =  76.08684973423297
Calibrated age =  8230.5
calibrated age error (-,+) = 196.5 127.5
------------
----60----
corrected age =  7507.1635690286585
c14 age uncertainty =  69.20410899997015
Calibrated age =  8283.0
calibrated age error (-,+) = 102.0 131.0
------------

In [ ]:

# choose your own save path
filename = "HOLSEA_2021_Singapore_14c"
savepath = f'/Users/rogercreel/Desktop/{filename}.xlsx'

df_c.to_excel(savepath)